In [1]:
import os
import torch
import numpy as np
import librosa 
import opensmile
from transformers import Wav2Vec2Processor, Wav2Vec2Model, Wav2Vec2ForCTC
from utilz import *

In [2]:
torch.cuda.is_available()

True

In [3]:
# from datasets import load_dataset

# dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
# dataset = dataset.sort("id")
# sampling_rate = dataset.features["audio"].sampling_rate

In [4]:
y, sr = librosa.load('../SIMS/Raw/video_0001/0001.mp4', sr=16000) # ffmpeg

/home/chason/anaconda3/envs/deepeye/lib/python3.10/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


# librosa 

In [5]:
mel_spec = librosa.feature.melspectrogram(y,sr,n_mels=128).T
print(mel_spec.shape) # 25ms

(41, 128)


/tmp/ipykernel_17758/417018464.py:1: FutureWarning: Pass y=[ 0.          0.          0.         ... -0.04392989 -0.04456997
 -0.05147557], sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel_spec = librosa.feature.melspectrogram(y,sr,n_mels=128).T


# Opensmile

In [6]:
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.ComParE_2016,
    feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
)

sml_fs = smile.process_file('../SIMS/Raw/video_0001/0001.mp4')
print(sml_fs.shape)
# 10ms

SoX could not be found!

    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    
/bin/sh: 1: sox: not found


(124, 65)


# Wav2Vec

In [7]:
# English
# MELD
# processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
# model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")

In [8]:
# Chinese
processor = Wav2Vec2Processor.from_pretrained("ydshieh/wav2vec2-large-xlsr-53-chinese-zh-cn-gpt")
model = Wav2Vec2ForCTC.from_pretrained("ydshieh/wav2vec2-large-xlsr-53-chinese-zh-cn-gpt")

/home/chason/anaconda3/envs/deepeye/lib/python3.10/site-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
# inputs = processor(dataset[0]["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")

inputs = processor(y, sampling_rate=sr, return_tensors="pt")
with torch.no_grad():
    # outputs = model(**inputs) # for EN
    outputs = model.wav2vec2(**inputs)
    
# last_hidden_states = outputs.last_hidden_state  # for EN
last_hidden_states = outputs.extract_features # for CH
list(last_hidden_states.shape)

2022-10-03 20:05:43.072207: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-03 20:05:43.170851: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-03 20:05:43.188666: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-03 20:05:43.575592: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

[1, 63, 512]

# Batch Processing

In [10]:
def get_speech_feature(file, mode, max_len=128):
    if mode == 'mel':
        y, sr = librosa.load(file)
        mel_spec = librosa.feature.melspectrogram(y, sr, n_mels=128).T # (time_steps, 128) (100,128)
        if len(word_embs)<max_len:
            mel_spec = np.concatenate([mel_spec, np.zeros((max_len-len(mel_spec), 128))], axis=0)
        return mel_spec[:max_len] #(128,128)
    
    elif mode == 'opensmile':
        smile = opensmile.Smile(feature_set=opensmile.FeatureSet.ComParE_2016,
                                feature_level=opensmile.FeatureLevel.LowLevelDescriptors)
        sml_fs = smile.process_file(file)
        if len(sml_fs)<max_len*2: 
            sml_fs = np.concatenate([sml_fs, np.zeros((max_len*2-len(sml_fs), 65))], axis=0)
        return sml_fs[:max_len*2] #(256,65)
    
    elif mode == 'wav2vec':
        y, sr = librosa.load(file, sr=16000)
        inputs = processor(y, sampling_rate=sr, return_tensors="pt")
        with torch.no_grad():
            outputs = model.wav2vec2(**inputs)
        wav2vec_fs = outputs.extract_features[0].cpu().detach().numpy() # for CH
        print(type(wav2vec_fs))
        if len(wav2vec_fs)<max_len: 
            wav2vec_fs = np.concatenate([wav2vec_fs, np.zeros((max_len-len(wav2vec_fs), 512))], axis=0)
        return wav2vec_fs[:max_len] #(128,512)
        
    

In [11]:
MODE = 'wav2vec'

video_path = '../SIMS/Raw/'
video_ids, clip_ids, texts, annotations, modes = load_data('../SIMS/label.csv')

acoustic = {'train':[], 'valid':[], 'test':[]}
labels = {'train':[], 'valid':[], 'test':[]}
label_dict = {'Negative':0, 'Positive':1, 'Neutral':2}
    

for video_id, clip_id, annotation, mode in zip(video_ids, clip_ids, annotations, modes):
    clip_id_ = '000' + str(clip_id)
    file_path = video_path + str(video_id) + '/' + clip_id_[-4:] + '.mp4' 
    
    print(file_path, mode)    
    
    acoustic[mode].append(get_speech_feature(file_path, mode=MODE))
    labels[mode].append(label_dict[annotation])
    
save_features(acoustic, './data/acoustic_wav2vec.pkl')
save_features(labels, './data/labels.pkl')

../SIMS/Raw/video_0001/0001.mp4 train


/home/chason/anaconda3/envs/deepeye/lib/python3.10/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


<class 'numpy.ndarray'>
../SIMS/Raw/video_0001/0002.mp4 train


/home/chason/anaconda3/envs/deepeye/lib/python3.10/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


<class 'numpy.ndarray'>
../SIMS/Raw/video_0001/0003.mp4 test


...

/home/chason/anaconda3/envs/deepeye/lib/python3.10/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


<class 'numpy.ndarray'>
../SIMS/Raw/video_0001/0016.mp4 train


/home/chason/anaconda3/envs/deepeye/lib/python3.10/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


<class 'numpy.ndarray'>
../SIMS/Raw/video_0001/0017.mp4 test


/home/chason/anaconda3/envs/deepeye/lib/python3.10/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)

KeyboardInterrupt

